In [1]:
import sys
from collections import namedtuple
from ortools.constraint_solver import pywrapcp

In [2]:
Set = namedtuple("Set", ['index', 'cost', 'items'])

def parse_data(input_data):
    # parse the input
    lines = input_data.split('\n')

    parts = lines[0].split()
    item_count = int(parts[0])
    set_count = int(parts[1])

    sets = []
    for i in range(1, set_count+1):
        parts = lines[i].split()
        # WARNING: we changed the cast here from FLOAT to INT
        sets.append(Set(i-1, int(parts[0]), [int(p) for p in parts[1:]]))
        
    return sets

def parse_file(filename):
    with open(filename) as f:
        return parse_data(f.read())
    
def summary(x, solver):    
    print("Wall time: {} ms".format(solver.WallTime()))
    print(solver)
    if not solver.NextSolution():
        print("No solution found!")
        return    
    print("Solution:", " ".join(str(x[i].Value()) for i in range(len(x))))

In [3]:
def set_cover(sets):
    items = set()
    for s in sets:
        items.update(s.items)
    
    item_count = len(items)
    set_count = len(sets)
    
    solver = pywrapcp.Solver("set_cover")
    
    # Decision vars for selecting the set i
    x = [solver.IntVar(0, 1, "x_{:02d}".format(i)) for i in range(set_count)]
    c = [solver.IntConst(sets[i].cost, "c_{:02d}".format(i)) for i in range(set_count)]
        
    # Minimizes the cost of the sum of the selected sets.
    solver.Minimize(solver.Sum([x[i] * c[i] for i in range(set_count)]), 1)
        
    # Each item must be covered by at least one of the sets specified
    
    covered_by = [[] for _ in range(item_count)]
    for s in range(set_count):
        for item in sets[s].items:
            covered_by[item].append(x[s])
        
    for i in range(item_count):                
        solver.Add(solver.Max(covered_by[i]) == 1)
        
    # Dumb search:
    db = solver.Phase(x, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)
    solver.Solve(db)
    
    return x, solver

In [4]:
data = parse_file("data/sc_10000_8")
x, solver = set_cover(data)
summary(x, solver)

Wall time: 852 ms
Solver(name = "set_cover", state = OUTSIDE_SEARCH, branches = 9935, fails = 0, decisions = 9935, delayed demon runs = 0, var demon runs = 20935, normal demon runs = 515055, Run time = 852 ms)
Solution: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 